# SQL Alchemy - Working with some SQL

SQL is the language of choice for database systems and finds wide use in any type of business software and as well in geospatial data. In general, SQL decompoeses into multiple sublanguages
- The DDL Data Definiton Language in which a database is described by creating tables consisting of columns of certain data types
- The SQL Query Language for inserting, selecting, and updating data.

SQL queries often return tables themselves and, thus, nested SQL queries can be used.

However, SQL is as well tied to a few limitations when it comes to programming: creating valid SQL statements is not easy and commonly leads to security issues or complex escaping code and inefficiencies or even limitations on the amount and type of data that can be manipulated. Therefore, automating all these aspects is a common approach.

Therefore, object-oriented programming and relational databases form a good pair of tools and can be mapped with each other through an *Object-Relational Mapper (ORM)*. SQL alchemy is a python package that provides a flexible cross-platform library for accessing various databases and as well an ORM engine allowing us to treat data in databases as Python classes. 

In this tutorial, we quickly recap how classes are done in Python and then connect to a database and build an ORM persistence engine.

In [1]:
%pip install sqlalchemy

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------  2.1/2.1 MB 11.8 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 9.0 MB/s  0:00:00

   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   ---------------------------------------- 2/2 [sqlalchemy]

Note: you may need to restart the kernel to use


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\ge98siw\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


# Classes in Python
A class is simply defined by its name and contains both attributes and member functions which all take at least one argument called self in python and used to access the current instance of the class. The following example creates a class with one class attribute

In [2]:
class BaseClass:
    name="BaseClassName"


If you want to create instances (objects) of classes, you can use the identifier of the class as a function to create a variable of this type:

In [3]:
b = BaseClass()
print(b)
print(b.name)


BaseClassName


In object-oriented programming, inheritance is an important concept in which a sub-class of a class is conceptualized to inherit all functions and attributes from parents, maybe changing some of them and adding new ones, but typically not removing any of those. In addition, it is possible to convert ("view") specific classes as instances of the base class in program code.

At the same time, we add a member function.

In [4]:
class Derived(BaseClass):
    name="Derived" # overwritten
    def member(self):
        print("Member Function can access itself through self:", self)
        print("My name is " + self.name)

d = Derived()
d.member()


Member Function can access itself through self: <__main__.Derived object at 0x000001F34AE5D6A0>
My name is Derived


# Object-Relational Mapping with SQLAlchemy

In object-relational mapping, not all apsects of a class are stored in the database. Only attributes can be stored there, not the functions, and in SQLAlchemy it is well possible that classes have some attributes mapped to the database (such that changing them changes the data in the database with the advantage of durability and the downside of storage overhead) and attributes that are not mapped to the database (e.g., information mainly for use in the program)

In contrast to SQL, where a database is designed by explicitly writing Data Definition Langauge (e.g., CREATE TABLE,...), SQLAlchemy allows us to specify a database just from Python code. 

- Therefore, attributes are just covered by Mapped as below.
- Special attributes (e.g., those that shall be primary keys) are mapped with the mapped_column function allowing us to specify such information

## SQL Alchemy - First Steps using SQL
Below example is derived from the official SQLAlchemy tutorial which contains many more details: https://docs.sqlalchemy.org/en/20/tutorial/index.html
using simple SQL queries to create a table, fill it with data, retrieve data, modify data, and delete data. In a way, a minimal data round trip using plain SQL language and an SQLite in-memory backend (note that you could use a file to keep the data over a longer time)


In [1]:
from sqlalchemy import create_engine, text
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True) # a database engine showing all queries
with engine.begin() as conn:
    conn.execute(text('CREATE TABLE primes (value INT);'))
    for q in [1,2,3,5,7]:
        conn.execute(text("INSERT INTO primes(value) VALUES (%d);"%(q)))
# Now, we have run a few inserts and have a table of prime numbers. The connection is gone. In a real database, data would have been committed.

# Query
with engine.begin() as conn:
    result = conn.execute(text('SELECT * FROM primes;')) # result is a database cursor that is Python-iterable 
    for r in result:
        print("Value found: %d"% (r.value)) # access by column name

# Query with WHERE
with engine.begin() as conn:
    result = conn.execute(text('SELECT * FROM primes WHERE value > 4;')) # result is a database cursor that is Python-iterable 
    for r in result:
        print("Value found: %d"% (r.value)) # access by column name

# Delete 1 as it is not a prime number
with engine.begin() as conn:
    conn.execute(text('DELETE FROM primes WHERE value=1'))

# Query again
with engine.begin() as conn:
    result = conn.execute(text('SELECT * FROM primes;')) # result is a database cursor that is Python-iterable 
    for r in result:
        print("Value found: %d"% (r.value)) # access by column name

# An update: 7=>11
with engine.begin() as conn:
    result = conn.execute(text('UPDATE primes SET  value=11 WHERE value=7;')) # result is a database cursor that is Python-iterable 


# Query again
with engine.begin() as conn:
    result = conn.execute(text('SELECT * FROM primes;')) # result is a database cursor that is Python-iterable 
    for r in result:
        print("Value found: %d"% (r.value)) # access by column name



2025-12-09 10:09:00,384 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-09 10:09:00,385 INFO sqlalchemy.engine.Engine CREATE TABLE primes (value INT);
2025-12-09 10:09:00,386 INFO sqlalchemy.engine.Engine [generated in 0.00050s] ()
2025-12-09 10:09:00,387 INFO sqlalchemy.engine.Engine INSERT INTO primes(value) VALUES (1);
2025-12-09 10:09:00,387 INFO sqlalchemy.engine.Engine [generated in 0.00057s] ()
2025-12-09 10:09:00,388 INFO sqlalchemy.engine.Engine INSERT INTO primes(value) VALUES (2);
2025-12-09 10:09:00,389 INFO sqlalchemy.engine.Engine [generated in 0.00097s] ()
2025-12-09 10:09:00,390 INFO sqlalchemy.engine.Engine INSERT INTO primes(value) VALUES (3);
2025-12-09 10:09:00,391 INFO sqlalchemy.engine.Engine [generated in 0.00048s] ()
2025-12-09 10:09:00,392 INFO sqlalchemy.engine.Engine INSERT INTO primes(value) VALUES (5);
2025-12-09 10:09:00,392 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ()
2025-12-09 10:09:00,394 INFO sqlalchemy.engine.Engine INSERT INTO pri

## SQL-Alchemy - Creating Python Classes for the Object Relational Mapper
The following snippet creates a few tables from Python classes extended with Schema information. Note that all of those Python classes only contain table metadata and are not linked to any database yet, but below, we will emit the creation SQL to an in-memory engine. Afterwards, we can use Sessions to access the database in terms of below class hierarchy

In [2]:
from sqlalchemy.orm import DeclarativeBase
class Base(DeclarativeBase):
    pass

from typing import List
from typing import Optional
from sqlalchemy import String, ForeignKey
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy import MetaData

metadata_obj = MetaData()

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"


sandy = User(name="sandy", fullname="Sandy Cheeks")
print(sandy)

User(id=None, name='sandy', fullname='Sandy Cheeks')


The previous class model is actually a database model as well. Now, we can emit DDL to the databse automatically as follows:


In [3]:
engine = create_engine("sqlite+pysqlite:///users.db", echo=True) # a database engine showing all queries
Base.metadata.create_all(engine) # This uses the DeclarativeBase to find all mapped tables

2025-12-09 10:09:09,130 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-09 10:09:09,131 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2025-12-09 10:09:09,132 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-09 10:09:09,133 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2025-12-09 10:09:09,134 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-09 10:09:09,135 INFO sqlalchemy.engine.Engine COMMIT


### SQL Alchmey - Manipulating Data

In [4]:
from sqlalchemy.orm import Session
session = Session (engine)
#session.add(sandy)
#session.flush()

In [11]:
print(sandy)

User(id=1, name='sandy', fullname='Sandy Cheeks')


In [7]:
sandy.name = "Sandie"
session.flush()
session.commit()


2025-12-09 10:08:35,201 INFO sqlalchemy.engine.Engine UPDATE user_account SET name=? WHERE user_account.id = ?
2025-12-09 10:08:35,202 INFO sqlalchemy.engine.Engine [generated in 0.00138s] ('Sandie', 1)
2025-12-09 10:08:35,203 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
from sqlalchemy import select
# Show all people in the database
res = session.execute(select(User))
for r in res:
    print(r)
    

2025-12-09 10:13:59,558 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2025-12-09 10:13:59,559 INFO sqlalchemy.engine.Engine [cached since 263.9s ago] ()
(User(id=1, name='Sandie', fullname='Sandy Cheeks'),)
(User(id=2, name='Martin', fullname='Martin Werner'),)


# Full Stack Database Applications Guideline
It is comparably easy to implement even complex applications using an SQL-compliant database and SQL alchemy. However, a certain pattern should be known and followed in order to minimize problems and challenges throughout the process. For a simple database application, we should have one file (maybe later more) defining the database and its mapping to python (Section 1) followed by all user-operations (Section 2) such that all operations can be executed on the command line.

When all user-operations are plain Python functions, we can use the fire Python library to provide a complete and pretty advanced command line interface. It is now a question of taste whether the database configuration is part of the source file, given on the command line, or given in a file. We will show the variant in which it is just hard-coded as this is sufficient even for many production environments.

However, we leave Jupyter Notebook for the command line and continue with the Python file dbex.py.

Librar